In [7]:
import requests

url = "https://passis-bfd9b877f7d0.herokuapp.com/"

table_list = ['memorias', 'recuperar_lista_documentos']

payload = {}
headers = {}


for l in table_list:
    n_url = url+l
    response = requests.request("GET", n_url, headers=headers, data=payload)
    print(response.text)



[{"content": "\ud83d\udcddperguntas para gartner: \n- percentual de empresas investindo em fine tuning \n- divis\u00e3o de estrat\u00e9gia entre foundation models, domain models and model hubs.", "date_created": "2023-09-23 17:37:47", "id": 2}, {"content": "\ud83d\udcdd esse \u00e9 meu primeiro registro de mem\u00f3ria", "date_created": "2023-09-23 15:22:08", "id": 1}]
{"documentos": [{"data_de_upload": "2023-09-10 00:26:46", "descricao": "Identidade civil ou Registro Geral\r\nN\u00famero do documento: MG10192099", "id": 5, "nome_do_documento": "RG", "versao": 1}, {"data_de_upload": "2023-09-10 00:31:07", "descricao": "Passaporte n\u00famero: GD208356\r\nData de expedi\u00e7\u00e3o: 05/11/2021\r\nValidade: 04/11/2031", "id": 6, "nome_do_documento": "Passaporte", "versao": 1}]}
